In [30]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ---------------------------------------- 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 11.1 MB/s eta 0:00:01
   ---------------------- ----------------- 1.4/2.5 MB 17.6 MB/s eta 0:00:01
   ---------------------------------------  2.5/2.5 MB 17.4 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 16.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/19.3 MB ? eta -:--:--
   - -------------------------------------- 0.5/19.3 MB 34.8 MB/s eta 0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.32.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.5 which is incompatible.
streamlit 1.32.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.1.2 which is incompatible.


In [32]:
!pip install python-dotenv

In [34]:
from dotenv import load_dotenv
import os 
load_dotenv()
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_SECRET_KEY")
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_SECRET_KEY")

In [36]:
#the documents 
question = "What kind of pets do I like?"
document = "My favorite pet is a dog."

In [38]:
#returning number of tokens in a text string 
import tiktoken 
def num_tokens_from_string(string: str, encoding_name: str):
    #getting the encoder/tokenizer object
    encoding = tiktoken.get_encoding(encoding_name)
    #encoding the string into to tokens, and then figuring out how many tokens there are 
    num_tokens = len(encoding.encode(string))
    return num_tokens 
#getting the number of tokens in the question 
num_tokens_from_string(question, "cl100k_base")
#can understand how large the documents we want to feed in are 

8

In [40]:
#getting a fixed length vector representation for both the document and the query/question 
from langchain_openai import OpenAIEmbeddings
embd = OpenAIEmbeddings()
query_result = embd.embed_query(question)
document_result = embd.embed_query(document)
#getting the length of the vector representation for the query
len(query_result)

1536

In [42]:
#cosine similairty is used to compare how similair to vectors are 
#it is done by calculating cosine of the angle between them 

In [44]:
import numpy as np
def cosine_similairty(vec1, vec2): 
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2) 
similairity = cosine_similairty(query_result, document_result)
print(similairity)

0.88133716682745


In [46]:
#loading in a document 
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [52]:
#now we are going to be splitting the document 
from langchain.text_splitter import RecursiveCharacterTextSplitter 
#configuring the splits 
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    #maximum size for each token 
    chunk_size = 300, 
    #each chunk will overlap with the previous one by 50 tokens
    chunk_overlap = 50)


#actually splitting 
splits = text_splitter.split_documents(blog_docs)

In [ ]:
from langchain_community.vectorstores import Chroma 
#taking each split, embedding it with OpenAI embeddings into a vector representation
#storing it in the vectorstore 
vectorstores = Chroma.from_documents(documents=splits, embeddings = OpenAIEmbeddings())
#turning the vectorstore into a an retriever object 
#will retrieve the most relevant document chunks based on the similairty score
#k:1 in the search_kwards means that retrieve only the top 1 most relevant document based on the cosine similairty 
retriever = vectorstores.as_retriever(search_kwargs={"k": 1})

In [ ]:
#example of getting relevant documents 
#feeding in a certain question too
docs = retriever.get_relevant_documents("What is task deocomposition?")

In [ ]:
from langchain_openai import ChatOpenAI 
from langchain_prompts import ChatPromptTemplate 
#this is the template which will be used to guide the llm 
#the question should be answered based on the given context only nothing outside of that 
template = """Answer the question based only on the following context:
{context}

Question: {qustion}
"""

#now we can create an actual LangChain ChatPromptTemplate obejct based off the string 
#this will make it structured